In [34]:
import requests
import json
import io
import numpy as np
import re

## Insert data

In [39]:
ts1 = '{"1": {"ts": {"times": [1, 2, 3], "values": [2, 4, 9]}}}'
ts2 = '{"2": {"ts": {"times": [1, 2, 3], "values": [1, 5, 10]}}}'

In [40]:
r = requests.post('http://127.0.0.1:5000/insert', json = ts1)
r = requests.post('http://127.0.0.1:5000/insert', json = ts2)

## Request data

In [41]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

In [42]:
print(r.text)

{
    "1": {},
    "2": {}
}


In [43]:
r = requests.get('http://127.0.0.1:5000/select?fields=ts', auth=('user', 'pass'))
print(r.text)

{
    "1": {
        "ts": {
            "values": [
                2,
                4,
                9
            ],
            "times": [
                1,
                2,
                3
            ]
        }
    },
    "2": {
        "ts": {
            "values": [
                1,
                5,
                10
            ],
            "times": [
                1,
                2,
                3
            ]
        }
    }
}


## Insert meta

In [49]:
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = '{"1": {"blarg": 1, "order": 1}}')
r = requests.post('http://127.0.0.1:5000/upsert_meta', json = '{"2": {"blarg": 2, "order": 2}}')

In [50]:
r = requests.get('http://127.0.0.1:5000/select?fields=order', auth=('user', 'pass'))
print(r.text)

<pre>{
    "1": {
        "order": 1
    },
    "2": {
        "order": 2
    }
}</pre>


## Sort by order

In [66]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Dorder', auth=('user', 'pass'))
print(r.text)

<pre>{
    "2": {},
    "1": {}
}</pre>


In [67]:
r = requests.get('http://127.0.0.1:5000/select?additional=sort_by,%2Border,limit,1', auth=('user', 'pass'))
print(r.text)

<pre>{
    "1": {}
}</pre>


## Filter by meta

In [106]:
r = requests.get('http://127.0.0.1:5000/select?md=blarg,<2', auth=('user', 'pass'))
# {'blarg': {'<=': 2}}
print(r.text)

<pre>{
    "1": {}
}</pre>


## Select augmented

In [107]:
augmented = {'md': {"blarg": {'>=': 1}},'proc':'stats','target':['damean','dastd'], 'additional':{"sort_by": "+order", "limit": 2}}
augemented_json = io.StringIO()
json.dump(augmented, fp=augemented_json)
augmented_data = augemented_json.getvalue()
augmented_data

'{"target": ["damean", "dastd"], "md": {"blarg": {">=": 1}}, "additional": {"limit": 2, "sort_by": "+order"}, "proc": "stats"}'

In [109]:
r = requests.post('http://127.0.0.1:5000/augmented_select', json = augmented_data)
# print (r.text)
print(r.text)

{
    "2": {
        "dastd": 3.6817870057290873,
        "damean": 5.333333333333333
    },
    "1": {
        "dastd": 2.943920288775949,
        "damean": 5.0
    }
}


## Delete ts

In [114]:
requests.post('http://127.0.0.1:5000/delete', json = '1')

<Response [200]>

In [111]:
r = requests.get('http://127.0.0.1:5000/select', auth=('user', 'pass'))

In [113]:
print(r.text)

<pre>{
    "2": {}
}</pre>


In [ ]:
requests.post('http://127.0.0.1:5000/delete', json = '2')

## Find similar

In [115]:
values = np.random.randint(1,100,100)
times = np.arange(1,101)
k_nearest = 3

In [116]:
ts =  {'times': times.tolist(), 'values': values.tolist(), 'k_nearest':k_nearest}

#  db.upsert_meta('one', {'order': 1, 'blarg': 1})
# print(meta)
ts_a = io.StringIO()
json.dump( ts, fp=ts_a)
ts_a.getvalue()


'{"k_nearest": 3, "values": [97, 29, 75, 11, 81, 75, 14, 17, 12, 47, 34, 50, 76, 38, 25, 98, 32, 25, 92, 63, 3, 52, 27, 50, 54, 48, 86, 35, 11, 63, 11, 70, 29, 97, 59, 23, 94, 84, 21, 61, 91, 30, 21, 35, 17, 34, 1, 49, 66, 92, 88, 61, 70, 57, 83, 35, 91, 45, 63, 15, 85, 11, 36, 33, 58, 2, 49, 25, 27, 93, 55, 86, 78, 45, 53, 87, 15, 51, 78, 41, 1, 22, 41, 45, 58, 79, 26, 81, 20, 27, 97, 12, 8, 27, 61, 52, 80, 77, 84, 62], "times": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]}'

In [117]:
r = requests.post('http://127.0.0.1:5000/find_similar', json = ts_a.getvalue())

In [118]:
print(r.text)

["ts-0", "ts-1", "ts-10"]
